# Day 4

In [1]:
import scala.collection.mutable.Map
import scala.util.matching.Regex
import java.time.LocalDateTime
import java.time.format.DateTimeFormatter

import scala.collection.mutable.Map

import scala.util.matching.Regex

import java.time.LocalDateTime

import java.time.format.DateTimeFormatter

In [2]:
val input = """[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up""".split("\n").toList

input: List[String] = List(
  "[1518-11-01 00:00] Guard #10 begins shift",
  "[1518-11-01 00:05] falls asleep",
  "[1518-11-01 00:25] wakes up",
  "[1518-11-01 00:30] falls asleep",
  "[1518-11-01 00:55] wakes up",
  "[1518-11-01 23:58] Guard #99 begins shift",
  "[1518-11-02 00:40] falls asleep",
  "[1518-11-02 00:50] wakes up",
  "[1518-11-03 00:05] Guard #10 begins shift",
  "[1518-11-03 00:24] falls asleep",
  "[1518-11-03 00:29] wakes up",
  "[1518-11-04 00:02] Guard #99 begins shift",
  "[1518-11-04 00:36] falls asleep",
  "[1518-11-04 00:46] wakes up",
  "[1518-11-05 00:03] Guard #99 begins shift",
  "[1518-11-05 00:45] falls asleep",
  "[1518-11-05 00:55] wakes up"
)

In [3]:
def sortByTime(a:(LocalDateTime, String), b:(LocalDateTime, String)) = {
    a._1.isAfter(b._1)
}

def sortList(times:List[String], 
             sorted_times:List[(LocalDateTime, String)]=Nil): List[(LocalDateTime, String)] = times match {
    case t::ts => {
        val pattern:Regex = "\\[([\\d\\-: ]+)\\] ([\\w# ]+)".r
        pattern.findFirstMatchIn(t) match {
            case Some(x) => {
                sortList(ts, (LocalDateTime.parse(x.group(1), DateTimeFormatter.ofPattern("yyyy-MM-dd HH:mm")), 
                                                x.group(2)) :: sorted_times)
            }
            case None => println("something has gone wrong"); Nil
        
        }
    }
    case Nil => sorted_times.sortWith(sortByTime).reverse
}

defined function sortByTime
defined function sortList

In [4]:
def sortBySleep(a:(Int, Int), b:(Int, Int)) = {
    a._2 > b._2
}

def findSleepyGuard(times:List[(LocalDateTime, String)], guards:Map[Int, Int]=Map()): Int = times match {
    case t::f::w::ts => {
//         println(t, f, w)
        val pattern:Regex = "Guard #(\\d+)".r
        pattern.findFirstMatchIn(t._2) match {
            case Some(x) => {
                if (f._2.contains("Guard")) {
                    // doesn't fall asleep
                    findSleepyGuard(f::w::ts, guards)
                } else {
                    var id:Int = x.group(1).toInt
                    var length_in_minutes:Int = w._1.getMinute - f._1.getMinute
                    if (guards.contains(id)) {
                        guards(id) += length_in_minutes 
                    } else {
                        guards(id) = length_in_minutes
                    }
                    if (ts != Nil && ts.head._2 == "falls asleep") findSleepyGuard(t::ts, guards) else findSleepyGuard(ts, guards)
                }
            }
            case _ => println(t, f, w); 0
        }
        
    }
    case _ => guards.toSeq.sortWith(sortBySleep)(0)._1
}

defined function sortBySleep
defined function findSleepyGuard

In [5]:
def findSleepiestMinute(id:Int, times:List[(LocalDateTime, String)], minutes:Map[Int,Int]=Map()): (Int, Int) = times match {
    case t::f::w::ts => {
        val pattern:Regex = "Guard #(\\d+)".r
        pattern.findFirstMatchIn(t._2) match {
            case Some(x) if (x.group(1).toInt == id) => {
                if (f._2.contains("Guard")) {
                    // doesn't fall asleep
                    findSleepiestMinute(id, f::w::ts, minutes)
                } else {
                    for (i <- f._1.getMinute until w._1.getMinute) {
                        if (minutes.contains(i)) minutes(i) += 1 else minutes(i) = 1
                    }
                    if (ts != Nil && ts.head._2 == "falls asleep") {
                        findSleepiestMinute(id, t::ts, minutes) 
                    } else {
                        findSleepiestMinute(id, ts, minutes)
                    }
                }
            }
            case _ => {
                if (ts != Nil && ts.head._2 == "falls asleep") {
                    findSleepiestMinute(id, t::ts, minutes) 
                } else {
                    findSleepiestMinute(id, ts, minutes)
                }
            }
        }
        
    }
    case _ => {
        if (minutes.size > 0) {
            minutes.toSeq.sortWith(sortBySleep)(0)
        } else {
            (0,0)
        }
    }
}

defined function findSleepiestMinute

In [6]:
var sorted_list = sortList(input)

sorted_list: List[(LocalDateTime, String)] = List(
  (1518-11-01T00:00, "Guard #10 begins shift"),
  (1518-11-01T00:05, "falls asleep"),
  (1518-11-01T00:25, "wakes up"),
  (1518-11-01T00:30, "falls asleep"),
  (1518-11-01T00:55, "wakes up"),
  (1518-11-01T23:58, "Guard #99 begins shift"),
  (1518-11-02T00:40, "falls asleep"),
  (1518-11-02T00:50, "wakes up"),
  (1518-11-03T00:05, "Guard #10 begins shift"),
  (1518-11-03T00:24, "falls asleep"),
  (1518-11-03T00:29, "wakes up"),
  (1518-11-04T00:02, "Guard #99 begins shift"),
  (1518-11-04T00:36, "falls asleep"),
  (1518-11-04T00:46, "wakes up"),
  (1518-11-05T00:03, "Guard #99 begins shift"),
  (1518-11-05T00:45, "falls asleep"),
  (1518-11-05T00:55, "wakes up")
)

In [7]:
var guard = findSleepyGuard(sorted_list)
findSleepiestMinute(guard, sorted_list)._1 * guard

guard: Int = 10
res6_1: Int = 240

## Part 2

In [8]:
def findGuards(times:List[(LocalDateTime, String)], guards:List[Int]=Nil):List[Int] = times match {
    case t::ts => {
        val pattern:Regex = "Guard #(\\d+)".r
        pattern.findFirstMatchIn(t._2) match {
            case Some(x) => {
                var id = x.group(1).toInt
                if (!guards.contains(id)) findGuards(ts, id::guards) else findGuards(ts, guards)
            }
            case _ => findGuards(ts, guards)
        }
        
    }
    case Nil => guards
}

defined function findGuards

In [9]:
def sortByFreq(a:(Int,(Int,Int)), b:(Int,(Int,Int))) = {
    a._2._2 > b._2._2
}

def findSleepyMinutes(guards:List[Int], times:List[(LocalDateTime, String)], 
                      sleepy_minutes:Map[Int,(Int,Int)]=Map()): Int = guards match {
    case g::gs => {
        val (minute, freq) = findSleepiestMinute(g, times)
        if (freq > 0) sleepy_minutes(g) = (minute, freq)
        findSleepyMinutes(gs, times, sleepy_minutes)
    }
    case Nil => {
        val t = sleepy_minutes.toSeq.sortWith(sortByFreq)(0)
        t._1 * t._2._1
    }
}

defined function sortByFreq
defined function findSleepyMinutes

In [10]:
val guards = findGuards(sorted_list)

guards: List[Int] = List(99, 10)

In [11]:
findSleepyMinutes(guards, sorted_list)

res10: Int = 4455